# APDFT Analysis

Predictions can use

- **QC**: HF, CCSD, or CCSD(T) energy differences with the species of interests or target,
- **APDFT**: HF, CCSD, or CCSD(T) energy differences by changing the nuclear charge of other systems that have the same number of electrons as the target,
- **APDFT*n***: Taylor series approximation with order *n* of the APDFT potential energy surface.

In [1]:
import numpy as np
import pandas as pd

from apdft_tools.utils import hartree_to_ev, all_dimer_systems
from apdft_tools.data import prepare_dfs
from apdft_tools.prediction import get_qc_change_charge_dimer, get_apdft_change_charge_dimer
from apdft_tools.analysis import apdft_error_change_charge_dimer

json_path = '../../json-data/dimer-pyscf.apdft-data.posthf.json'
df_qc, df_apdft = prepare_dfs(json_path, get_CBS=False)

## APDFT prediction errors

There is some intrinsic error in modeling a target system (e.g., N atom) by changing the nuclear charge of a reference system's basis set (e.g., C<sup> &ndash;</sup> ).
The following cell computes this error, and represents the best performance for APDFT without fortuitous errors.

In [2]:
system_label = 'o.h'
delta_charge = -1
target_initial_charge = 0  # Initial charge of the system.
basis_set = 'cc-pV5Z'
lambda_specific_atom = 0
change_signs = True  # Multiple all predictions by negative one (e.g., for electron affinities)

n_points = 2
poly_order = 4

In [3]:
use_fin_diff = False  # Use finite differences with Taylor series for APDFT predictions.
remove_outliers = False

ea_qc_prediction = get_qc_change_charge_dimer(
    df_qc, system_label, delta_charge,
    target_initial_charge=target_initial_charge,
    change_signs=change_signs, basis_set=basis_set,
    n_points=n_points, poly_order=poly_order, remove_outliers=remove_outliers
)
ea_apdft_predictions = get_apdft_change_charge_dimer(
    df_qc, df_apdft, system_label, delta_charge,
    target_initial_charge=target_initial_charge,
    change_signs=change_signs, basis_set=basis_set,
    use_fin_diff=use_fin_diff, lambda_specific_atom=lambda_specific_atom,
    n_points=n_points, poly_order=poly_order, remove_outliers=remove_outliers
)

ea_qc_prediction = hartree_to_ev(ea_qc_prediction)
ea_apdft_predictions = {key:hartree_to_ev(value) for (key,value) in ea_apdft_predictions.items()}
ea_apdft_errors = {key:value-ea_qc_prediction for (key,value) in ea_apdft_predictions.items()}

print(f'PySCF prediction of IP for {system_label}: {ea_qc_prediction:.3f} eV\n')
print(f'APDFT prediction errors in eV:')
print(pd.DataFrame(ea_apdft_errors, index=[f'APDFT']))

PySCF prediction of IP for o.h: 1.523 eV

APDFT prediction errors in eV:
           ne.h       f.h
APDFT  5.976645  1.141323


## APDFT*n* prediction errors

Now, we can look at approximating the APDFT prediction by using a Taylor series centered on $\Delta Z = 0$.

In [4]:
use_fin_diff = True  # Use finite differences with Taylor series for APDFT predictions.
remove_outliers = False

ea_qc_prediction = get_qc_change_charge_dimer(
    df_qc, system_label, delta_charge,
    target_initial_charge=target_initial_charge,
    change_signs=change_signs, basis_set=basis_set,
    n_points=n_points, poly_order=poly_order, remove_outliers=remove_outliers
)
ea_apdft_predictions = get_apdft_change_charge_dimer(
    df_qc, df_apdft, system_label, delta_charge,
    target_initial_charge=target_initial_charge,
    change_signs=change_signs, basis_set=basis_set,
    use_fin_diff=use_fin_diff, lambda_specific_atom=lambda_specific_atom,
    n_points=n_points, poly_order=poly_order, remove_outliers=remove_outliers
)

ea_qc_prediction = hartree_to_ev(ea_qc_prediction)
ea_apdft_predictions = {key:hartree_to_ev(value) for (key,value) in ea_apdft_predictions.items()}
ea_apdft_errors = {key:value-ea_qc_prediction for (key,value) in ea_apdft_predictions.items()}

print(f'PySCF prediction of IP for {system_label}: {ea_qc_prediction:.3f} eV\n')
print(f'APDFTn prediction errors in eV:')
print(pd.DataFrame(ea_apdft_errors, index=[f'APDFT{n}' for n in range(5)]))

PySCF prediction of IP for o.h: 1.523 eV

APDFTn prediction errors in eV:
             ne.h        f.h
APDFT0  29.667766  14.568295
APDFT1 -11.567414  -2.749806
APDFT2   0.953872  -0.067438
APDFT3   0.778176  -0.178758
APDFT4  -1.915091 -12.184128


/home/alex/.local/lib/python3.8/site-packages/apdft_tools/prediction.py:609: RankWarning: Polyfit may be poorly conditioned
  bond_lengths_fit, poly_coeffs = fit_dimer_poly(
/home/alex/.local/lib/python3.8/site-packages/apdft_tools/prediction.py:609: RankWarning: Polyfit may be poorly conditioned
  bond_lengths_fit, poly_coeffs = fit_dimer_poly(


### Specifying lambda values

We can also specify specific lambda values to include. For example, we could only look at lambda values of +-1.

In [5]:
considered_lambdas = [-1]
use_fin_diff = True  # Use finite differences with Taylor series for APDFT predictions.
remove_outliers = False

ea_qc_prediction = get_qc_change_charge_dimer(
    df_qc, system_label, delta_charge,
    target_initial_charge=target_initial_charge,
    change_signs=change_signs, basis_set=basis_set,
    n_points=n_points, poly_order=poly_order, remove_outliers=remove_outliers
)
ea_apdft_predictions = get_apdft_change_charge_dimer(
    df_qc, df_apdft, system_label, delta_charge,
    target_initial_charge=target_initial_charge,
    change_signs=change_signs, basis_set=basis_set,
    use_fin_diff=use_fin_diff, lambda_specific_atom=lambda_specific_atom,
    n_points=n_points, poly_order=poly_order, remove_outliers=remove_outliers,
    considered_lambdas=considered_lambdas
)

ea_qc_prediction = hartree_to_ev(ea_qc_prediction)
ea_apdft_predictions = {key:hartree_to_ev(value) for (key,value) in ea_apdft_predictions.items()}
ea_apdft_errors = {key:value-ea_qc_prediction for (key,value) in ea_apdft_predictions.items()}

ea_qc_prediction = hartree_to_ev(ea_qc_prediction)
ea_apdft_predictions = {key:hartree_to_ev(value) for (key,value) in ea_apdft_predictions.items()}
ea_apdft_errors = {key:value-ea_qc_prediction for (key,value) in ea_apdft_predictions.items()}

print(f'PySCF prediction of IP for {system_label}: {ea_qc_prediction:.3f} eV\n')
print(f'APDFTn prediction errors in eV:')
print(pd.DataFrame(ea_apdft_errors, index=[f'APDFT{n}' for n in range(5)]))

PySCF prediction of IP for o.h: 41.431 eV

APDFTn prediction errors in eV:
               f.h
APDFT0  396.423493
APDFT1  -74.826024
APDFT2   -1.835086
APDFT3   -4.864248
APDFT4 -331.546997


/home/alex/.local/lib/python3.8/site-packages/apdft_tools/prediction.py:609: RankWarning: Polyfit may be poorly conditioned
  bond_lengths_fit, poly_coeffs = fit_dimer_poly(


## APDFT*n* errors with respect to APDFT

Or you, can compute the difference between APDFT*n* (predictions with Taylor series) and APDFT.

In [6]:
compute_difference = True
use_fin_diff = True  # Use finite differences with Taylor series for APDFT predictions.
remove_outliers = False

ea_apdft_predictions = get_apdft_change_charge_dimer(
    df_qc, df_apdft, system_label, delta_charge,
    target_initial_charge=target_initial_charge,
    change_signs=change_signs, basis_set=basis_set,
    use_fin_diff=use_fin_diff, lambda_specific_atom=lambda_specific_atom,
    n_points=n_points, poly_order=poly_order, remove_outliers=remove_outliers,
    compute_difference=compute_difference
)

ea_apdft_predictions = {key:hartree_to_ev(value) for (key,value) in ea_apdft_predictions.items()}

print(f'Differences between APDFTn and APDFT in eV:')
print(pd.DataFrame(ea_apdft_predictions, index=[f'APDFT{n}' for n in range(5)]))

Differences between APDFTn and APDFT in eV:
             ne.h        f.h
APDFT0  31.190339  16.090869
APDFT1 -10.044840  -1.227232
APDFT2   2.476446   1.455135
APDFT3   2.300750   1.343816
APDFT4  -0.392518 -10.661554


/home/alex/.local/lib/python3.8/site-packages/apdft_tools/prediction.py:609: RankWarning: Polyfit may be poorly conditioned
  bond_lengths_fit, poly_coeffs = fit_dimer_poly(
/home/alex/.local/lib/python3.8/site-packages/apdft_tools/prediction.py:609: RankWarning: Polyfit may be poorly conditioned
  bond_lengths_fit, poly_coeffs = fit_dimer_poly(


## Overall statistics

We can also compute mean absolute errors (MAEs), root mean squared error (RMSE) and max error.

In [7]:
all_systems = all_dimer_systems[0:-2]
target_initial_charge = 0

considered_lambdas = None
compute_difference = False
use_fin_diff = True  # Use finite differences with Taylor series for APDFT predictions.
remove_outliers = False
max_apdft_order = 4

for i in range(len(all_systems)):
    sys_error = apdft_error_change_charge_dimer(
        df_qc, df_apdft, all_systems[i], delta_charge,
        target_initial_charge=target_initial_charge,
        change_signs=change_signs, basis_set=basis_set,
        use_fin_diff=use_fin_diff, lambda_specific_atom=lambda_specific_atom,
        n_points=n_points, poly_order=poly_order, remove_outliers=remove_outliers,
        compute_difference=compute_difference, considered_lambdas=considered_lambdas
    )
    if i == 0:
        all_error = sys_error
    else:
        all_error = pd.concat(
            [all_error, sys_error], axis=1
        )

if use_fin_diff or compute_difference == True:
    # MAE
    for n in range(0, max_apdft_order+1):
        apdftn_errors = all_error.iloc[n].values
        apdftn_mae = np.mean(np.abs(apdftn_errors))
        print(f'APDFT{n} MAE: {apdftn_mae:.4f} eV')

    # RMSE
    print()
    for n in range(0, max_apdft_order+1):
        apdftn_errors = all_error.iloc[n].values
        apdftn_rmse = np.sqrt(np.mean((apdftn_errors)**2))
        print(f'APDFT{n} RMSE: {apdftn_rmse:.4f} eV')
    
    # Max
    print()
    for n in range(0, max_apdft_order+1):
        apdftn_errors = all_error.iloc[n].values
        apdftn_max = np.max(np.abs(apdftn_errors))
        print(f'APDFT{n} max abs.: {apdftn_max:.4f} eV')
else:
    # MAE
    apdftn_errors = all_error.iloc[0].values
    apdftn_mae = np.mean(np.abs(apdftn_errors))
    print(f'APDFT MAE: {apdftn_mae:.4f} eV')

    # RMSE
    print()
    apdftn_rmse = np.sqrt(np.mean((apdftn_errors)**2))
    print(f'APDFT RMSE: {apdftn_rmse:.4f} eV')

    # Max
    print()
    apdftn_max = np.max(np.abs(apdftn_errors))
    print(f'APDFT max abs.: {apdftn_max:.4f} eV')


/home/alex/.local/lib/python3.8/site-packages/apdft_tools/prediction.py:609: RankWarning: Polyfit may be poorly conditioned
  bond_lengths_fit, poly_coeffs = fit_dimer_poly(
/home/alex/.local/lib/python3.8/site-packages/apdft_tools/prediction.py:609: RankWarning: Polyfit may be poorly conditioned
  bond_lengths_fit, poly_coeffs = fit_dimer_poly(
/home/alex/.local/lib/python3.8/site-packages/apdft_tools/prediction.py:609: RankWarning: Polyfit may be poorly conditioned
  bond_lengths_fit, poly_coeffs = fit_dimer_poly(
/home/alex/.local/lib/python3.8/site-packages/apdft_tools/prediction.py:609: RankWarning: Polyfit may be poorly conditioned
  bond_lengths_fit, poly_coeffs = fit_dimer_poly(
/home/alex/.local/lib/python3.8/site-packages/apdft_tools/prediction.py:609: RankWarning: Polyfit may be poorly conditioned
  bond_lengths_fit, poly_coeffs = fit_dimer_poly(
/home/alex/.local/lib/python3.8/site-packages/apdft_tools/prediction.py:609: RankWarning: Polyfit may be poorly conditioned
  bond

APDFT0 MAE: 18.7703 eV
APDFT1 MAE: 6.7099 eV
APDFT2 MAE: 1.5233 eV
APDFT3 MAE: 5.9243 eV
APDFT4 MAE: 18.2325 eV

APDFT0 RMSE: 20.3500 eV
APDFT1 RMSE: 7.8401 eV
APDFT2 RMSE: 2.2097 eV
APDFT3 RMSE: 9.1494 eV
APDFT4 RMSE: 46.6459 eV

APDFT0 max abs.: 29.6678 eV
APDFT1 max abs.: 11.5674 eV
APDFT2 max abs.: 4.6207 eV
APDFT3 max abs.: 18.0475 eV
APDFT4 max abs.: 159.4453 eV


/home/alex/.local/lib/python3.8/site-packages/apdft_tools/prediction.py:609: RankWarning: Polyfit may be poorly conditioned
  bond_lengths_fit, poly_coeffs = fit_dimer_poly(
/home/alex/.local/lib/python3.8/site-packages/apdft_tools/prediction.py:609: RankWarning: Polyfit may be poorly conditioned
  bond_lengths_fit, poly_coeffs = fit_dimer_poly(
/home/alex/.local/lib/python3.8/site-packages/apdft_tools/prediction.py:609: RankWarning: Polyfit may be poorly conditioned
  bond_lengths_fit, poly_coeffs = fit_dimer_poly(
/home/alex/.local/lib/python3.8/site-packages/apdft_tools/prediction.py:609: RankWarning: Polyfit may be poorly conditioned
  bond_lengths_fit, poly_coeffs = fit_dimer_poly(
/home/alex/.local/lib/python3.8/site-packages/apdft_tools/prediction.py:609: RankWarning: Polyfit may be poorly conditioned
  bond_lengths_fit, poly_coeffs = fit_dimer_poly(
/home/alex/.local/lib/python3.8/site-packages/apdft_tools/prediction.py:609: RankWarning: Polyfit may be poorly conditioned
  bond